In [8]:
import numpy as np
np.random.seed(1337)

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Lambda, Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout, Activation
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
from sklearn import metrics
import time

Using Theano backend.


In [48]:
context_train = []
ad_train = []
y_train = []
with open('../../data/click-train') as f:
    for line in f:
        label, context, ad = line.strip().split("\t")
        y_train.append(int(label))
        context_train.append(map(lambda x: int(x), context.split(" ")))
        ad_train.append(map(lambda x: int(x), ad.split(" ")))
        
        
context_test = []
ad_test = []
y_test = []
with open('../../data/click-test') as f:
    for line in f:
        label, context, ad = line.strip().split("\t")
        y_test.append(int(label))
        context_test.append(map(lambda x: int(x), context.split(" ")))
        ad_test.append(map(lambda x: int(x), ad.split(" ")))

In [49]:
max_features = 2 ** 17
maxlen = 15


context_train = sequence.pad_sequences(context_train, maxlen=maxlen)
ad_train = sequence.pad_sequences(ad_train, maxlen=maxlen)

context_test = sequence.pad_sequences(context_test, maxlen=maxlen)
ad_test = sequence.pad_sequences(ad_test, maxlen=maxlen)



In [50]:
def means(x):
    return K.mean(x, axis=1)

In [61]:
from keras.layers import Merge

batch_size = 32
embedding_dims = 1


left_branch = Sequential()
left_branch.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
# left_branch.add(BatchNormalization())
left_branch.add(Dropout(0.5))
left_branch.add(Lambda(means, output_shape=(embedding_dims,)))
# left_branch.add(Dense(32, input_dim=embeding_dim))

right_branch = Sequential()
right_branch.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
# left_branch.add(BatchNormalization())
left_branch.add(Dropout(0.5))
right_branch.add(Lambda(means, output_shape=(embedding_dims,)))
# right_branch.add(Dense(32, input_dim=embeding_dim))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)

final_model.add(Dense(20))
final_model.add(BatchNormalization())
final_model.add(LeakyReLU(alpha=.01))
final_model.add(Dense(1))
final_model.add(BatchNormalization())
final_model.add(Activation('sigmoid'))

final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['fmeasure'])

model_path = ''
for i in range(10):
    
    
    if model_path != '':
        final_model.load_weights(model_path)
    model_path = '../../model/weights.epoch' + str(i) + "_" + str(int(time.time())) + '.hdf5'
    weight_save_callback = ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto')
    final_model.fit([context_train, ad_train], y_train, verbose=2, batch_size=batch_size,validation_data=([context_test, ad_test], y_test), nb_epoch=1, callbacks=[weight_save_callback])
    
    y_pred = final_model.predict([context_train, ad_train])[:, 0]
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred)
    print "train auc -> " + str(metrics.auc(fpr, tpr))
    
    y_pred = final_model.predict([context_test, ad_test])[:, 0]
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    print "test auc -> " + str(metrics.auc(fpr, tpr))



Train on 10000 samples, validate on 10000 samples
Epoch 1/1
8s - loss: 0.7766 - fmeasure: 0.4847 - val_loss: 0.6945 - val_fmeasure: 0.0000e+00
train auc -> 0.679091303289
test auc -> 0.52214441479
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
8s - loss: 0.6997 - fmeasure: 0.5383 - val_loss: 0.6921 - val_fmeasure: 0.1746
train auc -> 0.786231809687
test auc -> 0.528855109055
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
8s - loss: 0.6406 - fmeasure: 0.6266 - val_loss: 0.6924 - val_fmeasure: 0.4535
train auc -> 0.854723230508
test auc -> 0.529542391735
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
8s - loss: 0.6115 - fmeasure: 0.6719 - val_loss: 0.7707 - val_fmeasure: 0.6367
train auc -> 0.88129955095
test auc -> 0.5315305559
Train on 10000 samples, validate on 10000 samples
Epoch 1/1
8s - loss: 0.5860 - fmeasure: 0.6992 - val_loss: 0.7799 - val_fmeasure: 0.6253
train auc -> 0.892371218897
test auc -> 0.525873803112
Train on 10000 samples, valida

In [6]:
# final_model.load_weights('../../model/weights.01-0.69.hdf5')
# final_model.fit([context_train, ad_train], y_train, verbose=2, batch_size=batch_size,validation_data=([context_test, ad_test], y_test), nb_epoch=2, callbacks=[weight_save_callback])

In [7]:
from keras.layers import Merge

batch_size = 32
embedding_dims = 100


left_branch = Sequential()
left_branch.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
# left_branch.add(BatchNormalization())
left_branch.add(Dropout(0.5))
left_branch.add(Lambda(means, output_shape=(embedding_dims,)))
# left_branch.add(Dense(32, input_dim=embeding_dim))

right_branch = Sequential()
right_branch.add(Embedding(input_dim=max_features, output_dim=embedding_dims, init='glorot_uniform'))
# left_branch.add(BatchNormalization())
left_branch.add(Dropout(0.5))
right_branch.add(Lambda(means, output_shape=(embedding_dims,)))
# right_branch.add(Dense(32, input_dim=embeding_dim))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)

final_model.add(Dense(20))
final_model.add(BatchNormalization())
final_model.add(LeakyReLU(alpha=.01))
final_model.add(Dense(1))
final_model.add(BatchNormalization())
final_model.add(Activation('sigmoid'))

final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['fmeasure'])


from keras.utils.visualize_util import plot
plot(model, to_file='model.png')

RuntimeError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [ ]:
# load model to predict
model_path = '../../model/weights.epoch3_1481833706.hdf5'
final_model.load_weights(model_path)

y_pred = final_model.predict([context_train, ad_train])[:, 0]
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred)
print "train auc -> " + str(metrics.auc(fpr, tpr))

y_pred = final_model.predict([context_test, ad_test])[:, 0]
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
print "test auc -> " + str(metrics.auc(fpr, tpr))

In [ ]:
! ls ../../modelweights.epoch3_1481833706.hdf5

In [9]:
wer

NameError: name 'wer' is not defined